<a href="https://colab.research.google.com/github/masaguaro/llms-deep-dive-tutorials/blob/main/tutorials/chapter2/understanding_llms_and_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation and Imports

In [1]:
!pip install datasets transformers[sentencepiece,torch]
!pip install apache_beam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from datasets import load_dataset, DatasetDict

from transformers import (
    BertTokenizer,
    BertForMaskedLM,
    GPT2Tokenizer,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    AutoConfig,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)

## Understanding Masked LM's

In [3]:
## The first model we will look at is BERT, which is trained with masked tokens. As an example,
## the text below masks the word "box" from a well-known movie quote.

text = "Life is like a [MASK] of chocolates."

In [4]:
## We'll now see how BERT is able to predict the missing word. We can use HuggingFace to load
## a copy of the pretrained model and tokenizer.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [5]:
## Next, we'll feed our example text into the tokenizer.

encoded_input = tokenizer(text, return_tensors='pt')
print('input_ids:', encoded_input['input_ids'])
print('attention_mask:', encoded_input['attention_mask'])

input_ids: tensor([[ 101, 2166, 2003, 2066, 1037,  103, 1997, 7967, 2015, 1012,  102]])
attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [6]:
## input_ids represents the tokenized output. Each integer can be mapped back to the corresponding string.

Id_list = [ 101, 2166, 2003, 2066, 1037,  103, 1997, 7967, 2015, 1012,  102]
print(tokenizer.decode([7967]))
print(tokenizer.decode([101]))
for item in Id_list:
  print(tokenizer.decode([item]))

chocolate
[CLS]
[CLS]
life
is
like
a
[MASK]
of
chocolate
##s
.
[SEP]


In [7]:
## The model will then receive the output of the tokenizer. We can look at the BERT model to see exactly how
## it was constructed and what the outputs will be like.

model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [8]:
## The model starts with an embedding of each of the 30,522 possible tokens into 768 dimensions, which at this
## point is simply a representation of each token without any additional information about their relationships
## to one another in the text. Then the encoder attention blocks are applied, updating the embeddings such that
## they now encode each token's contribution to the chunk of text and interactions with other tokens. Notably,
## this includes the masked tokens as well. The final stage is the language model head, which takes the embeddings
## from the masked positions back to 30,522 dimensions. Each index of this final vector corresponds to the
## probability that the token in that position would be the correct choice to fill the mask.


model_output = model(**encoded_input)
output = model_output["logits"]

print(output.shape)

torch.Size([1, 11, 30522])


In [9]:
tokens = encoded_input['input_ids'][0].tolist()
masked_index = tokens.index(tokenizer.mask_token_id)
logits = output[0, masked_index, :]

print(logits.shape)

torch.Size([30522])


In [10]:
probs = logits.softmax(dim=-1)
values, predictions = probs.topk(5)
sequence = tokenizer.decode(predictions)

print('Top 5 predictions:', sequence)
print(values)

Top 5 predictions: box bag bowl jar cup
tensor([0.1764, 0.1688, 0.0419, 0.0336, 0.0262], grad_fn=<TopkBackward0>)


Printing the top 5 predictions and their respective scores, we see that BERT accurately chooses "box" as the most likely replacement for the mask token.

## Understanding Causal LM's

In [11]:
## We now repeat a similar exercise with the causal LLM GPT-2. This model generates
## text following an input, instead of replacing a mask within the text.

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
## We can examine the model again, noting the similarities to BERT. An embedding, 12 attention blocks,
## and a linear transformation bringing the output back to the size of the tokenizer. The tokenizer is
## different from BERT so we see we have more tokens this time.

model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [13]:
## We'll use a different text example, since this model works by producing tokens sequentially
## rather than filling a mask.

text = "Swimming at the beach is"
model_inputs = tokenizer(text, return_tensors='pt')
model_inputs

{'input_ids': tensor([[10462, 27428,   379,   262, 10481,   318]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [14]:
## After applying the model, the information needed to predict the next token is represented by
## the last token. So we can access that vector by the index -1.

output = model(**model_inputs)
next_token_logits = output.logits[:, -1, :]
next_token = torch.argmax(next_token_logits, dim=-1)
print(next_token)

tensor([257])


In [15]:
## Now add the new token to the end of the text, and feed all of it back to the model to continue
## predicting more tokens.

model_inputs['input_ids'] = torch.cat([model_inputs['input_ids'], next_token[:, None]], dim=-1)
model_inputs["attention_mask"] = torch.cat([model_inputs['attention_mask'], torch.tensor([[1]])], dim=-1)
print(model_inputs)

{'input_ids': tensor([[10462, 27428,   379,   262, 10481,   318,   257]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [16]:
## Here's what we have so far. The model added the word 'a' to the input text.

print(tokenizer.decode(model_inputs['input_ids'][0]))

Swimming at the beach is a


In [17]:
## Repeating all the previous steps, we then add the word 'great'.

output = model(**model_inputs)
next_token_logits = output.logits[:, -1, :]
next_token = torch.argmax(next_token_logits, dim=-1)
model_inputs['input_ids'] = torch.cat([model_inputs['input_ids'], next_token[:, None]], dim=-1)
model_inputs["attention_mask"] = torch.cat([model_inputs['attention_mask'], torch.tensor([[1]])], dim=-1)
print(tokenizer.decode(model_inputs['input_ids'][0]))

Swimming at the beach is a great


In [18]:
## HuggingFace automates this iterative process. We'll use the quicker approach to finish our sentence.

output_generate = model.generate(**model_inputs, max_length=20, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output_generate[0]))

Swimming at the beach is a great way to get a little extra energy.

The beach


## Pre-training a GPT-2 model from scratch

Next we'll train a GPT-2 model from scratch using English Wikipedia data. Note that we're only using a tiny subset of the data to demonstrate that the model is capable of learning. The exact same approach could be followed on the full dataset to train a more functional model, but that would require a lot of compute.

In [19]:
dataset = load_dataset("wikipedia", "20220301.en")
ds_shuffle = dataset['train'].shuffle()

raw_datasets = DatasetDict(
    {
        "train": ds_shuffle.select(range(50)),
        "valid": ds_shuffle.select(range(50, 100))
    }
)

raw_datasets

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-00041.parquet:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

train-00001-of-00041.parquet:   0%|          | 0.00/705M [00:00<?, ?B/s]

...

train-00039-of-00041.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

train-00040-of-00041.parquet:   0%|          | 0.00/185M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6458670 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 50
    })
    valid: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 50
    })
})

In [20]:
print(raw_datasets['train'][0]['text'][:200])

Romeo Toogood ARCA HRUA (6 May 1902- 11 August 1966) was an Ulster artist and teacher who specialized in landscape painting.

Early life 
Romeo Charles Toogood was born in Belfast on 6 May 1902. He wa


In [21]:
## We'll tokenize the text, setting the context size to 128 and thus breaking each document into chunks of 128 tokens.

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("gpt2")

outputs = tokenizer(
    raw_datasets["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 24
Input chunk lengths: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 36, 128, 128, 15]
Chunk mapping: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


In [22]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 179
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 291
    })
})

Now we can set up the HuggingFace Trainer as follows. Since we're using such a small dataset, we'll need lots of epochs for the model to make progress because all of the parameters are randomly initialized at the outset. Typically, most LLM's are trained for only one epoch and more diverse examples.

In [23]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)

In [25]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [26]:
args = TrainingArguments(
    output_dir="wiki-gpt2",
    evaluation_strategy="steps",
    num_train_epochs=100
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-26-78b3b47f657b>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [27]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

The training loss is low by the end, which means the model should perform very well on training examples it has seen. It does not generalize well to the validation set of course, since we deliberately overfit on a small train set.

We can confirm with a couple of examples that were seen in training.

In [ ]:
text = tokenizer.decode(tokenized_datasets["train"][0]['input_ids'][:16])
print(text)

In [ ]:
model_inputs = tokenizer(text, return_tensors='pt')
print(model_inputs['input_ids'].shape)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_inputs['input_ids'] = model_inputs['input_ids'].to(device)
model_inputs['attention_mask'] = model_inputs['attention_mask'].to(device)

output_generate = model.generate(**model_inputs, max_new_tokens=16)
output_generate

In [ ]:
sequence = tokenizer.decode(output_generate[0])
print(sequence)

The model should do quite well at reciting text after seeing it so many times. We can be convinced that the tokenizer, model architecture, and training objective are well-suited to learning Wikipedia data. For comparison, we'll try this model on text from the validation set.

In [ ]:
text = tokenizer.decode(tokenized_datasets["valid"][0]['input_ids'][:32])
print(text)

In [ ]:
model_inputs = tokenizer(text, return_tensors='pt')

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_inputs['input_ids'] = model_inputs['input_ids'].to(device)
model_inputs['attention_mask'] = model_inputs['attention_mask'].to(device)

output_generate = model.generate(**model_inputs, max_new_tokens=16)
sequence = tokenizer.decode(output_generate[0])
print(sequence)

In [ ]:
raw_datasets['valid'][0]['text']

As expected, our model is completely confused this time. We'd need to train for much longer, and on much more diverse data, before we would have a model that can sensibly complete prompts it has never seen before. This is precisely why pre-training is such an important and powerful technique. If we had to train on all of Wikipedia for every NLP application to achieve optimal performance, it would be prohibitively expensive. But there's no need to do that when we can share and reuse existing pre-trained models as we did in the first part of this tutorial.